In [1]:
import pandas as pd
import numpy as np
import pickle
from joblib import dump
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:

time_precision = 4   # jagame päeva xh suurusteks plokkideks


X_train = pd.read_csv("../../data/X_train.csv")
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.ToimAeg = X_train.ToimAeg.values.astype('datetime64[D]')
X_train.ToimAeg = X_train['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_train["Aasta"] = X_train['ToimAeg'].dt.year
X_train["Kuu"] = X_train['ToimAeg'].dt.month
X_train["Päev"] = X_train['ToimAeg'].dt.day
X_train["Tund"] = X_train['ToimAeg'].dt.hour
X_train.drop(columns="ToimAeg", inplace=True)
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.head()

y_train = pd.read_csv("../../data/y_train.csv")
y_train = y_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_train.head()

X_valid = pd.read_csv("../../data/X_valid.csv")
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.ToimAeg = X_valid.ToimAeg.values.astype('datetime64[D]')
X_valid.ToimAeg = X_valid['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_valid["Aasta"] = X_valid['ToimAeg'].dt.year
X_valid["Kuu"] = X_valid['ToimAeg'].dt.month
X_valid["Päev"] = X_valid['ToimAeg'].dt.day
X_valid["Tund"] = X_valid['ToimAeg'].dt.hour
X_valid.drop(columns="ToimAeg", inplace=True)
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.head()

y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.head()

X_train.head()
y_train.info()
y_train.drop(columns=["Kahjusumma","Koht"], inplace=True)
y_valid.drop(columns=["Kahjusumma", "Koht"], inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62476 entries, 0 to 62475
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LIIK_AVALIKU_KORRA_RIKKUMINE   62476 non-null  int8  
 1   LIIK_JALGRATTA_MOPEEDI_VARGUS  62476 non-null  int8  
 2   LIIK_KEHALINE_VAARKOHTLEMINE   62476 non-null  int8  
 3   LIIK_KELMUS                    62476 non-null  int8  
 4   LIIK_KONFLIKT/OMAVOLI          62476 non-null  int8  
 5   LIIK_KOOLIVAGIVALD             62476 non-null  int8  
 6   LIIK_LIIKLUSRIKKUMINE          62476 non-null  int8  
 7   LIIK_MOOTORSOIDUKI_VARGUS      62476 non-null  int8  
 8   LIIK_MUU                       62476 non-null  int8  
 9   LIIK_PISIVARGUS                62476 non-null  int8  
 10  LIIK_ROOVIMINE                 62476 non-null  int8  
 11  LIIK_TULEKAHJU                 62476 non-null  int8  
 12  LIIK_VANDALISM                 62476 non-null  int8  
 13  L

In [3]:
rfc = RandomForestClassifier(n_estimators=100, random_state=1).fit(X_train, y_train)

In [5]:
probs_rfc = rfc.predict(X_valid)
# predict_proba  jaoks df tegemine
#df_proba = pd.DataFrame(np.array([x.transpose()[1] for x in probs_rfc]).transpose(), columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud

# predict jaoks df tegemine
df_proba = pd.DataFrame(probs_rfc, columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud

In [7]:
for col in df_proba.columns:
    print(metrics.roc_auc_score(y_valid[col], df_proba[col]), col, sep="\t")

0.8036343614899151	LIIK_AVALIKU_KORRA_RIKKUMINE
0.5468188511545904	LIIK_JALGRATTA_MOPEEDI_VARGUS
0.5598555319747373	LIIK_KEHALINE_VAARKOHTLEMINE
0.5229982811180519	LIIK_KELMUS
0.4999356995884774	LIIK_KONFLIKT/OMAVOLI
0.705818184563353	LIIK_KOOLIVAGIVALD
0.5	LIIK_LIIKLUSRIKKUMINE
0.5127561627561626	LIIK_MOOTORSOIDUKI_VARGUS
0.5198204721040637	LIIK_MUU
0.7538831427029081	LIIK_PISIVARGUS
0.4997408655091993	LIIK_ROOVIMINE
0.5	LIIK_TULEKAHJU
0.5371776242743985	LIIK_VANDALISM
0.6854015352844804	LIIK_VARGUS


In [8]:
dump(rfc, "trained_models/rfc.joblib", compress=3, protocol=5)

['trained_models/rfc.joblib']